In [23]:
import re

import numpy as np
import pandas as pd
import nltk
# nltk.download()
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn import metrics
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
import re
import string

df = pd.read_csv('imdb_sup.csv')

In [3]:
#helper functions for cleaning
def remove_punc(word):
    punc = '''-!()[]{};:"\,<>./?@#$%^&*_~'''
    no_punc_word = word.translate(str.maketrans('', '', punc))
    return no_punc_word

def remove_BR(lst):
    words = []
    for word in lst:
        if word.endswith('br'):
            word = word[:-2]
        words.append(word)
    
    return ' '.join(words)

def make_lower(lst):
    no_upper_list = [x.lower() for x in lst]
    return ' '.join(no_upper_list)

def remove_numbers(lst):
    no_num_list = [x for x in lst if not any(c.isdigit() for c in x)]
    return ' '.join(no_num_list)

def remove_words(lst):
    extra_stops = ['aa', 'ab','br', 'us', 'mr', 'saw', 'until', 'no', 'when', 'with', 
            'like', 'just', 'even', 'it\'s', 'i\'m', 
            'who', 'i\'ve', 'what', 'he', 'see', 'up','get', 'been',
            'because', 'into', 'time', 'watch', 'â–','called', '2',
             '10', 'said','their', 'can','two', 'go', 'also', 'seen', 'him',
            'through', 'it', 'doesn\'t', 'you\'re', 'that\'s', 'there\'s',
            'come', 'said', 'all.', 'screen', 'person', 'i\'ll', 'is,'
            '5', 'sandra', 'them.', '3', '.', 'he\'s', 'man', 'they\'re',
            '\\\x96', '--', 'i\'d', 'is,', 'oh', 'one', 'much', 'movies',
            'say', '4', '1', 'five', 'what\'s', '15', 'ed', '...',
            'movie', 'film','', '-', 'people', 'could', 'make', 'films', 'reviews']


    stop_words = nltk.corpus.stopwords.words("english")
    stop_words = set(stop_words)
    stop_words.update(extra_stops) 

    words = [word for word in lst if word not in stop_words]

    return ' '.join(words)
    

In [4]:
def text_clean(message):
  message = remove_punc(message)   # remove punctiation except - and \'
  message = remove_words(message.split())
  message = remove_BR(message.split())
  message = remove_numbers(message.split())
  message = make_lower(message.split())
  message = nltk.tag.pos_tag(message.split(), tagset='universal')
  message = [word for word,tag in message if (tag == 'ADJ' or tag == 'ADV' or tag == 'VERB')]
  message = ' '.join(message)
  # message.lower()
  
  
  # print(message)
  

  return message

In [5]:
test_str = "Skippy A wonderful wonderous wonder from \"Family Ties\" plays Eddie, a wussy 'metal' nerd who gets picked on. When his favorite wussy 'metal' singer, Sammi Curr, dies, he throws a hissy fit tearing down all the posters on his bedroom wall. But when he later gets an unreleased record that holds the spirit of his dead 'metal' idol. He first gets sucked into ideas of revenge, but then he doesn't want to take it as far as Sammi does. Which isn't really that far as his main victims only seem to go to the hospital. This movie is utterly laughable and has about as much to do with real metal as say, \"Rock Star\". OK, maybe a tad more than that piece of junk, but you get my point. And how ANYone can root for a guy played by Skippy from \"Family Ties\" I haven't a clue. The cameo by Gene Simmons is OK, and Ozzy Osbourne reaches coherency, I applaud him for that, but otherwise skip this one.<br /><br />My Grade: D <br /><br />Eye Candy:Elise Richards gets topless, an a topless extra at a pool party"
text_clean(test_str)

'skippy wonderful wonderous wonder plays eddie wussy nerd gets picked when favorite dies throws hissy fit tearing bedroom later gets unreleased holds spirit dead first gets sucked revenge want take far really far main seem utterly laughable real ok maybe tad piece played skippy i clue ok ozzy osbourne reaches applaud otherwise skip gets topless extra'

In [6]:
#cleans all reviews in dataset
df['Review'] = df['Review'].apply(text_clean)

In [7]:
#shuffles data entries
dff = df.sample(frac=1).reset_index(drop=True)

In [8]:
dff

,Review,Rating,Sentiment
0,absolutely terrible funny snobby little treats...,1,0
1,alfred hitchcok favorite means imagine done ho...,4,0
2,i've several alice wonderland take worst bough...,1,0
3,may potential plausible enjoyable main roberts...,3,0
4,classic cop flick sadly overlooked basically f...,8,1
...,...,...,...
49995,i shorts leaps ahead acting directing filming ...,9,1
49996,unconditional best it's emotional pub when gra...,10,1
49997,first offwith fred ray janmichael vincent rhin...,4,0
49998,private spun fairly successful well written gr...,3,0


In [9]:
#stemming program / not used as of now - lowers accuracy 
porterstemmer = PorterStemmer()

def steming(message):
  return[porterstemmer.stem(word) for word in message.split()]

In [10]:
#dividing data betweeen test and tain
train_X = dff['Review'].head(25000)  
train_Y = dff['Sentiment'].head(25000)  
test_X = dff['Review'].tail(25000) 
test_Y = dff['Sentiment'].tail(25000)

In [11]:
#tf idf
tf_idf = TfidfVectorizer()
count = CountVectorizer()
#applying tf idf to training data
X_train_tf = tf_idf.fit_transform(train_X)
# count_X_train_tf = count.fit_transform(train_X)
#applying tf idf to training data


In [41]:
#bag or words - not super helpful given the size of our data but here's top 20 words - meaning they have the most features in documents
text_X = [review for review in train_X]

count_matrix = count.fit_transform(text_X)
count_array = count_matrix.toarray()
print(count_matrix.shape)


tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit(count_matrix)
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=coun_vect.get_feature_names_out(),columns=["idf_weights"])

#inverse document frequency
df_idf.sort_values(by=['idf_weights'])

# count matrix 
count_vector=coun_vect.transform(text_X) 
# tf-idf scores 
tf_idf_vector=tfidf_transformer.transform(count_vector)

feature_names = coun_vect.get_feature_names_out() 
#get tfidf vector for first document 
first_document_vector=tf_idf_vector[0] 
#print the scores 
df = pd.DataFrame(first_document_vector.T.todense(), index=feature_names, columns=["tfidf"]) 
df.sort_values(by=["tfidf"],ascending=False)

(25000, 66465)


,tfidf
snobby,0.317767
hispanic,0.279741
treats,0.275119
senses,0.259214
abc,0.249715
...,...
friendskill,0.000000
friendsloved,0.000000
friendsneighborsunite,0.000000
friendsso,0.000000


In [33]:
print("n_samples: %d, n_features: %d" % X_train_tf.shape)

n_samples: 25000, n_features: 66465


In [34]:
#transforming test data into tf-idf matrix
X_test_tf = tf_idf.transform(test_X)

print("n_samples: %d, n_features: %d" % X_test_tf.shape) # the result of the test data and training data are the same

n_samples: 25000, n_features: 66465


In [36]:
#naive bayes classifier
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tf, train_Y)
#predicted y
y_pred = naive_bayes_classifier.predict(X_test_tf)

print(metrics.classification_report(test_Y, y_pred, target_names=['neg','pos']))
print("Confusion matrix:")
print(metrics.confusion_matrix(test_Y, y_pred))

              precision    recall  f1-score   support

         neg       0.85      0.87      0.86     12516
         pos       0.87      0.85      0.86     12484

    accuracy                           0.86     25000
   macro avg       0.86      0.86      0.86     25000
weighted avg       0.86      0.86      0.86     25000

Confusion matrix:
[[10920  1596]
 [ 1887 10597]]


In [16]:
test = 'First of all it was my childhood dream to see these hero\'s in a big screen and it was full filled. The movie was quite good for me but after getting out of the theatre there were not much scenes in my mind that i can feel awesome about. Zack Snyder is a master in making comic book style of movies he has proved it by his movie the Watchmen but in the movie the Justice league the movie lost its synder touch and the whole movie looks like a rip off to marvel movie mantra that they use in there films(like the happy go jolly characters not much tension on the screen light hearted) likewise Josh wedon has made this movie into a marvel like film. The Justice league had many character moments for all but the grip was loosen up to make the fan/audience glue to there seats. The whole colour tone of the movie was changed by josh and as the Wb asked them to cut down its length to 2hrs(as the internet says Wb asked them to do so) But anyone who watched the film can easily see that many of the scenes showned in the trailer is not in the movie,even the dialogue by steppen wolf "No protector here.. .. .   .... This world will fall like all the others" that one was also cut off in the film.in justice league the expensive reshooted scenes that have no connection to the actual film plot and at first report the movies run time was nearly 3hrs and now we got a 2hrs 1min movie to watch with most of the scenes cut off..As a movie lover its just hurts know that they reduce the length of the film that has so much potential to reach for more.When Bvs released the critics and some marvel fan boys said its so dark and lengthy with not much strong plot to establish the cause of the fight between Batman and superman. But after they release the extended cut bvs we all get to know the exact chronology of the plot. Likewise this movie should also release the extended cut/directors cut version but zack was great for making something that is so gripping and his tones delivers the theme itself but here its all josh wedon who just ruined Zack\'s vision and made a marvel-istic movie with his Cuts. Overall the movie was Great and All the actors played there roles perfectly and the image of batman had be shifted abit in my view as there is a bit of difference between the Bvs batman and the JL batman, Bvs batman was more like the exact comic book Dark Knight but in this He is more Like Tony stark here and there. And those who relay on F**king tomatoes ratings don\'t watch If you are a movie lover and if this is your genre then go for it From the intro scene of Batman you will love the movie While watching the films you won\'t be able to see any flaws because of its paced 2hrs cut.For me the rating for this film would be 6.5/10. Go and watch for yourself its an entertaining Flick don\'t miss it ..the only draw back you will have will be with its running time and scenes cut off.'

testB = 'Within the first two minutes it was already clear that this film was gonna suck. Every shot of this film feels incredibly forced.  There was not one shot where I could forget I was watching a movie that had a crew with incredibly large egos.  The pacing, the shot choices, the action sequences,  the ridiculously contrived editing, the terrible soundtrack that was a mish mash of action movie devices without a good central theme, the dialogue, the acting, NOTHING FLOWED.  Even the wardrobe dept.  yes it’s a costume, but it’s not suppose to feel like a costume.  The lack of textures made everything feel like plastic.  The first scene where the poor Icelandic people were dressed like 1930s depression era fisherman was a bit insulting.  I feel I must apologize to Icelandic people for how they were portrayed in this film.  And every time this film tried to inject humanity it was so forced I had to swallow my vomit.  This is why films are not graphic novels.  Frankly it felt like the entire movie was just aggravating slow pans from from one still to another, before shots are gratuitously sped up to try and create some level of  adrenaline.  A lot of panning, oh so much panning, and when they were panning they were slow zooming.  And when they weren’t slow zooming, the camera was at a dramatic tilt.  I can’t say much about the acting because they direction was so excessive, that there was no room for actors.  Most of them just had to make sure they got their lines out and didn’t accidentally act too much as to break the very stiff composition of the shot.  Their faces were half shadowed in most scenes so you couldn’t see them anyway. . . one more turd in the DC universes.  Please stop letting Zack Snyder do anything. . . Just stop.'

testc = "I liked this movie. But I had a problem with the plot. It has my favorite actor."
processed = []
processed.append(text_clean(test))

In [17]:
test_input = tf_idf.transform(processed)
test_input.shape

#0= bad review
#1= good review
res=naive_bayes_classifier.predict(test_input)[0]

if res==1:
    print("Good Review")
elif res==0:
    print("Bad Review")

Good Review
